In [68]:
# importing all necessary libraries
from sqlalchemy import create_engine
import pymysql
import datetime

In [41]:
# Conneting to MySQL database
try:
    engine = create_engine('mysql+pymysql://<user_name>:<database_name>@<host_name>/<table>')
    print('Connected to Mysql Successfully')
except Exception as e:
    print('issue with connecting to Mysql',e)

Connected to Mysql Successfully


In [70]:
try:
    with engine.connect() as con:
        s3_url = 'en-fr.csv'
        i = 0
        skiprows = 0
        nrows = 100000
        count = 1000
        while count != 0:
            start_time = datetime.datetime.now()
            chunk = pd.read_csv(s3_url,skiprows = skiprows, nrows=nrows, names = ['en','fr'])
            print('chunk_{} read successfully'.format(i))
            chunk.dropna()
            chunk.to_sql('entofr',con, if_exists = 'append',index = False)
            end_time = datetime.datetime.now()
            count = chunk.shape[0]
            skiprows = skiprows + nrows
            i =+1
            print('chuck_{} inserted'.format(i))
            print('Time taken', end_time - start_time)
except Exception as e:
    print('Error while loading into mysql',e)

chunk_0 read successfully
chuck_1 inserted
Time taken 0:00:53.120820
chunk_1 read successfully
chuck_2 inserted
Time taken 0:01:01.790904
chunk_2 read successfully
chuck_3 inserted
Time taken 0:01:27.562634
chunk_3 read successfully
chuck_4 inserted
Time taken 0:01:00.113267
chunk_4 read successfully
chuck_5 inserted
Time taken 0:01:01.409209
chunk_5 read successfully
chuck_6 inserted
Time taken 0:00:55.355727
chunk_6 read successfully
chuck_7 inserted
Time taken 0:00:49.103079
chunk_7 read successfully
chuck_8 inserted
Time taken 0:00:57.504061
chunk_8 read successfully
chuck_9 inserted
Time taken 0:01:00.835258
chunk_9 read successfully
chuck_10 inserted
Time taken 0:01:08.216207
chunk_10 read successfully
chuck_11 inserted
Time taken 0:00:50.158370
chunk_11 read successfully
chuck_12 inserted
Time taken 0:00:52.472733
chunk_12 read successfully
chuck_13 inserted
Time taken 0:01:03.629537
chunk_13 read successfully
chuck_14 inserted
Time taken 0:00:58.012504
chunk_14 read successfull

chuck_116 inserted
Time taken 0:01:08.702071
chunk_116 read successfully
chuck_117 inserted
Time taken 0:01:06.328633
chunk_117 read successfully
chuck_118 inserted
Time taken 0:01:07.776395
chunk_118 read successfully
chuck_119 inserted
Time taken 0:01:07.143033
chunk_119 read successfully
chuck_120 inserted
Time taken 0:01:06.632573
chunk_120 read successfully
chuck_121 inserted
Time taken 0:01:16.010733
chunk_121 read successfully
chuck_122 inserted
Time taken 0:01:11.490065
chunk_122 read successfully
chuck_123 inserted
Time taken 0:01:11.011955
chunk_123 read successfully
chuck_124 inserted
Time taken 0:01:12.499661
chunk_124 read successfully
chuck_125 inserted
Time taken 0:01:16.415902
chunk_125 read successfully
chuck_126 inserted
Time taken 0:01:07.850875
chunk_126 read successfully
chuck_127 inserted
Time taken 0:01:07.745591
chunk_127 read successfully
chuck_128 inserted
Time taken 0:01:06.934565
chunk_128 read successfully
chuck_129 inserted
Time taken 0:01:06.932889
chunk_

KeyboardInterrupt: 

In [13]:
import redshift_connector

In [19]:
try:
    conn = redshift_connector.connect(
         host='redshift-cluster-2.ct7l6untnfxu.us-east-1.redshift.amazonaws.com',
         port=5439,
         database='dev',
         user='awsuser',
         password='RedShift1'
      )
    print('Successfully connected to Redshift')
except Exception as e:
    print('Unable to Connect to Redshift', e)

    

Successfully connected to Redshift


In [20]:
try:
    conn = redshift_connector.connect(
         host='redshift-cluster-2.ct7l6untnfxu.us-east-1.redshift.amazonaws.com',
         port=5439,
         database='dev',
         user='awsuser',
         password='RedShift1'
      )
    print('Successfully connected to Redshift')
except Exception as e:
    print('Unable to Connect to Redshift', e)

    

Successfully connected to Redshift


In [22]:
try:
    with conn.cursor() as cursor:
        cursor.execute("create table entofr(en varchar(1024), fr varchar(1024))")
        print('Table Created')
except Exception as e:
    print('Table not created')

Table Created


In [31]:
import mysql.connector

In [71]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [73]:
AWS_SERVER_PUBLIC_KEY = '<AWS_SERVER_PUBLIC_KEY>'
AWS_SERVER_SECRET_KEY = '<AWS_SERVER_SECRET_KEY>'
REGION_NAME = 'us-east-1'

In [119]:
s3 = boto3.Session().resource(service_name='s3',
    region_name=REGION_NAME,
    aws_access_key_id='<aws_access_key_id>',
    aws_secret_access_key='<aws_secret_access_key>')

In [84]:
try:
    s3.create_bucket(Bucket='entofrfolder')
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [147]:
prefix = 'split_files'
bucket_name = 'entofrfolder'
output_path ='s3://{}/{}'.format(bucket_name,prefix)

In [148]:
import os
if connect.is_connected():
    try:
        cursor = connect.cursor()
        cursor.execute('use translator')
        cursor.execute('select count(*) from entofr')
        limit = 100000
        offset = 0
        max_row_count = cursor.fetchall()[0][0]
        row_count = 0
        file_number = 0
        while row_count != max_row_count:
            cursor.execute('select * from entofr limit {} offset {}'.format(limit,offset))
            sample = cursor.fetchall()
            sample = pd.DataFrame(sample)
            sample.to_csv('entofr_{}.csv'.format(file_number),index=False)
            s3.Bucket(bucket_name).Object(os.path.join(prefix, '/entofr_{}.csv'.format(file_number))).upload_file('entofr_{}.csv'.format(file_number))
            print('entofr_{} is succesfully uploaded to s3'.format(file_number))
            file_number+=1 
            row_count += sample.shape[0]
            offset += 100000
    except Exception as e:
        print('Unsuccessful upload to s3', e)

entofr_0 is succesfully uploaded to s3
entofr_1 is succesfully uploaded to s3
entofr_2 is succesfully uploaded to s3
entofr_3 is succesfully uploaded to s3
entofr_4 is succesfully uploaded to s3
entofr_5 is succesfully uploaded to s3
entofr_6 is succesfully uploaded to s3
entofr_7 is succesfully uploaded to s3
entofr_8 is succesfully uploaded to s3
entofr_9 is succesfully uploaded to s3
entofr_10 is succesfully uploaded to s3
entofr_11 is succesfully uploaded to s3
entofr_12 is succesfully uploaded to s3
entofr_13 is succesfully uploaded to s3
entofr_14 is succesfully uploaded to s3
entofr_15 is succesfully uploaded to s3
entofr_16 is succesfully uploaded to s3
entofr_17 is succesfully uploaded to s3
entofr_18 is succesfully uploaded to s3
entofr_19 is succesfully uploaded to s3
entofr_20 is succesfully uploaded to s3
entofr_21 is succesfully uploaded to s3
entofr_22 is succesfully uploaded to s3
entofr_23 is succesfully uploaded to s3
entofr_24 is succesfully uploaded to s3
entofr_25 

KeyboardInterrupt: 